In [60]:
from __future__ import division

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import OneClassSVM
import pickle

In [61]:

# load the data
df_train = pd.read_csv('train.csv/train.csv')
df_test = pd.read_csv('test.csv/test.csv')

#load the pickle where extra computer features are stored
my_features = pickle.load(open("my_features.pickle", "rb"))

In [62]:
feature_to_clean = ['ind_var2_0', 'ind_var2', 'ind_var27_0', 'ind_var28_0', 'ind_var28', 'ind_var27', 'ind_var41', 'ind_var46_0', 'ind_var46', 'num_var27_0', 'num_var28_0', 'num_var28', 'num_var27', 'num_var41', 'num_var46_0', 'num_var46', 'saldo_var28', 'saldo_var27', 'saldo_var41', 'saldo_var46', 'imp_amort_var18_hace3', 'imp_amort_var34_hace3', 'imp_reemb_var13_hace3', 'imp_reemb_var33_hace3', 'imp_trasp_var17_out_hace3', 'imp_trasp_var33_out_hace3', 'num_var2_0_ult1', 'num_var2_ult1', 'num_reemb_var13_hace3', 'num_reemb_var33_hace3', 'num_trasp_var17_out_hace3', 'num_trasp_var33_out_hace3', 'saldo_var2_ult1', 'saldo_medio_var13_medio_hace3',
                   'ind_var29_0', 'ind_var29', 'ind_var13_medio', 'ind_var18', 'ind_var26', 
                     'ind_var25', 'ind_var32', 'ind_var34', 'ind_var37', 'ind_var39', 'num_var29_0', 
                     'num_var29', 'num_var13_medio', 'num_var18', 'num_var26', 'num_var25', 'num_var32', 
                     'num_var34', 'num_var37', 'num_var39', 'saldo_var29', 'saldo_medio_var13_medio_ult1', 
                     'delta_num_reemb_var13_1y3', 'delta_num_reemb_var17_1y3', 'delta_num_reemb_var33_1y3', 
                     'delta_num_trasp_var17_in_1y3', 'delta_num_trasp_var17_out_1y3', 
                     'delta_num_trasp_var33_in_1y3', 'delta_num_trasp_var33_out_1y3',
                   'delta_imp_reemb_var33_1y3',  'imp_reemb_var17_hace3', 'imp_reemb_var33_ult1', 'imp_trasp_var17_in_hace3', 'num_reemb_var17_hace3', 'num_reemb_var33_ult1', 'num_trasp_var17_in_hace3']



df_train.drop(feature_to_clean, axis=1, inplace=True)
df_test.drop(feature_to_clean, axis=1, inplace=True)

## Add Sum of 0's

In [63]:
df_train['sum_of_0'] = my_features['sum_of_0']

In [66]:
#prepare data for model
y_train = df_train['TARGET'].values
X_train = df_train.drop(['ID','TARGET'], axis=1).values

In [67]:
id_test = df_test['ID']
X_test = df_test.drop(['ID'], axis=1).values

In [68]:
# classifier
clf = xgb.XGBClassifier(missing=np.nan, max_depth=5, n_estimators=350, 
                        learning_rate=0.03, nthread=4, subsample=0.95, 
                        colsample_bytree=0.85, seed=4242)

In [69]:
X_fit, X_eval, y_fit, y_eval= train_test_split(X_train, y_train, test_size=0.3)

In [70]:
# fitting
clf.fit(X_train, y_train, early_stopping_rounds=20, 
        eval_metric="auc", eval_set=[(X_eval, y_eval)])

Will train until validation_0 error hasn't decreased in 20 rounds.
[0]	validation_0-auc:0.770236
[1]	validation_0-auc:0.783812
[2]	validation_0-auc:0.809264
[3]	validation_0-auc:0.794632
[4]	validation_0-auc:0.796601
[5]	validation_0-auc:0.800912
[6]	validation_0-auc:0.807597
[7]	validation_0-auc:0.811988
[8]	validation_0-auc:0.804085
[9]	validation_0-auc:0.803113
[10]	validation_0-auc:0.805332
[11]	validation_0-auc:0.803820
[12]	validation_0-auc:0.805995
[13]	validation_0-auc:0.803417
[14]	validation_0-auc:0.805842
[15]	validation_0-auc:0.807690
[16]	validation_0-auc:0.806464
[17]	validation_0-auc:0.809177
[18]	validation_0-auc:0.810439
[19]	validation_0-auc:0.807804
[20]	validation_0-auc:0.809302
[21]	validation_0-auc:0.810471
[22]	validation_0-auc:0.811616
[23]	validation_0-auc:0.812487
[24]	validation_0-auc:0.813940
[25]	validation_0-auc:0.815122
[26]	validation_0-auc:0.816580
[27]	validation_0-auc:0.817988
[28]	validation_0-auc:0.819187
[29]	validation_0-auc:0.821332
[30]	validati

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.85,
       gamma=0, learning_rate=0.03, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=350, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=4242, silent=True, subsample=0.95)

In [79]:
print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(X_train)[:,1]))

('Overall AUC:', 0.88301063305245064)


In [80]:
# predicting
y_pred= clf.predict_proba(X_test)[:,1]

In [81]:
y_pred

array([ 0.05264726,  0.06802616,  0.00226393, ...,  0.00463163,
        0.07819876,  0.00168039], dtype=float32)

In [82]:
submission = pd.DataFrame({"ID":id_test, "TARGET":y_pred})
submission.to_csv("submission16.csv", index=False)

In [75]:
'''
ADDITIONAL FUNCTIONS FOR FEATURE ENGINEERING
==============================================================
import pickle
to_save = {
    "sum_of_0":df_train['sum_of_0']
}
pickle.dump(to_save, open('my_features.pickle','a'))
===============================================================
#NAME: ADD SUM OF 0'S
# select on the features, leaving out the 'TARGET' feature


features = [f for f in df_train.columns if f != 'TARGET' and f!='ID' ]

def calculate_zeros(cols):
    sum_array = [] 
    for value in cols:
        if value == 0:
            sum_array.append(value)
    return len(sum_array)

df_train['sum_of_0'] = df_train[features].apply(calculate_zeros, axis=1)

===============================================================

# remove duplicated columns
remove = []
c = df_train.columns
for i in range(len(c)-1):
    v = df_train[c[i]].values
    for j in range(i+1, len(c)):
        if np.array_equal(v, df_train[c[j]].values):
            remove.append(c[j])

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

================================================================

# remove constant columns
remove = []
for col in df_train.columns:
    if df_train[col].std() == 0:
        remove.append(col)
================================================================

def fix_nationality(nat):
    for value in nat:
        if value == -999999:
            return 2
        else:
            return value

df_train['var3'] = df_train[['var3']].apply(fix_nationality, axis=1)


'''

'\nADDITIONAL FUNCTIONS FOR FEATURE ENGINEERING\n==============================================================\nimport pickle\nto_save = {\n    "sum_of_0":df_train[\'sum_of_0\']\n}\npickle.dump(to_save, open(\'my_features.pickle\',\'a\'))\n===============================================================\n#NAME: ADD SUM OF 0\'S\n# select on the features, leaving out the \'TARGET\' feature\n\n\nfeatures = [f for f in df_train.columns if f != \'TARGET\' and f!=\'ID\' ]\n\ndef calculate_zeros(cols):\n    sum_array = [] \n    for value in cols:\n        if value == 0:\n            sum_array.append(value)\n    return len(sum_array)\n\ndf_train[\'sum_of_0\'] = df_train[features].apply(calculate_zeros, axis=1)\n\n===============================================================\n\n# remove duplicated columns\nremove = []\nc = df_train.columns\nfor i in range(len(c)-1):\n    v = df_train[c[i]].values\n    for j in range(i+1, len(c)):\n        if np.array_equal(v, df_train[c[j]].values):\n        